In [1]:
from src.data_utils import load_xsum_dict, load_shuffled_test_split

xsum_test = load_xsum_dict("test")
bart_test_extrinsic = load_shuffled_test_split(xsum_test, "bart-test-extrinsic", len(xsum_test))

Using custom data configuration default
Reusing dataset xsum (/Users/anton164/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
from src.data_utils import load_summaries_from_logs

sums_by_id, sum_ents_by_id, failed_sums_by_id = load_summaries_from_logs(
    f"results/gef-logs/bart-full-classifier-knnv1.json",
    max_iterations=5,
)
len(sums_by_id)

11334

In [11]:
from src.detect_entities import detect_entities
from src.entity_utils import filter_entities
from src.entity_factuality import (
    ANNOTATION_LABELS,
)
from src.data_utils import get_gold_xsum_data
from src.oracle import oracle_label_entities, get_entity_annotations
from sumtool.storage import get_summaries

gold_sums, gold_metadata = get_gold_xsum_data()


for sum_id, summary in sums_by_id.items():
    if sum_id not in sum_ents_by_id:
        sum_ents_by_id[sum_id] = detect_entities(
            summary, xsum_test[sum_id]["document"]
        )
labeled_ents = oracle_label_entities(
    sum_ents_by_id,
    get_entity_annotations(sum_ents_by_id.keys(), gold_metadata),
    "strict_all",
)
unknown_entities = filter_entities(
    lambda x: x["label"] == ANNOTATION_LABELS["Unknown"],
    labeled_ents,
)

baseline_dataset = get_summaries("xsum", "facebook-bart-large-xsum")
baseline_sums_by_id = {sum_id: x["summary"] for sum_id, x in baseline_dataset.items()}
baseline_sum_ents_by_id = {}
for sum_id, summary in baseline_sums_by_id.items():
    if sum_id not in baseline_sum_ents_by_id:
        baseline_sum_ents_by_id[sum_id] = detect_entities(
            summary, xsum_test[sum_id]["document"]
        )

baseline_labeled_ents = oracle_label_entities(
    baseline_sum_ents_by_id,
    get_entity_annotations(sum_ents_by_id.keys(), gold_metadata),
    "strict_all",
)
baseline_unknown_entities = filter_entities(
    lambda x: x["label"] == ANNOTATION_LABELS["Unknown"],
    baseline_labeled_ents,
)

In [13]:
import json

xsum_extrinsic_fully_annotated = []
for sum_id, unknown_ents in unknown_entities.items():
    if len(unknown_ents) == 0 and len(baseline_unknown_entities[sum_id]) == 0 and sum_id in bart_test_extrinsic:
        xsum_extrinsic_fully_annotated.append(sum_id)

print(len(xsum_extrinsic_fully_annotated))
with open("./data/xsum_fully_annotated_test_splits.json", "w") as f:
    json.dump({
        "bart-extrinsic-fully-annotated": xsum_extrinsic_fully_annotated
    }, f)

139
